# Импорты

In [2]:
import numpy as np
import itertools

# Константы

In [3]:
EPS = 1e-4

# Проверка матрица на полную положительность и неотрицательность

In [12]:
def calc_order(n: int):
    for i in range(n):
        for j in range(i, n):
            yield i, j

        for j in range(i + 1, n):
            yield j, i


def check_minor(mat: np.ndarray, i: int, j: int):
    if i == j:
        minor = np.round(np.linalg.det(mat[:(i + 1), :(j + 1)]))
    elif j > i:
        minor = np.round(np.linalg.det(mat[:(i + 1), (j - i):(j + 1)]))
    else:
        minor = np.round(np.linalg.det(mat[(i - j):(i + 1), :(j + 1)]))

    if minor < EPS:
        return False

    return True


def total_positivity_test(mat: np.ndarray):
    n = mat.shape[0]

    for i, j in calc_order(n):
        if not check_minor(mat, i, j):
            return False

    return True

def findsubsets(s: set, n: int):
    return list(itertools.combinations(s, n))

def total_non_negativity_test(mat: np.ndarray):
    n = mat.shape[0]

    indices = set([i for i in range(n)])

    for k in range(n):
        subsets = findsubsets(indices, k + 1)

        for i in subsets:
            for j in subsets:
                submatrix = mat[i, :][:, j]

                if np.round(np.linalg.det(submatrix)) < 0:
                    return False

    return True

# Тесты

In [13]:
pos1 = np.array([[1, 2], [1, 3]])
pos2 = np.array([[6, 7], [8, 10]])
pos3 = np.array([[2, 4, 8], [3, 9, 27], [4, 16, 64]])

nonneg1 = np.array([[1, 1], [3, 3]])
nonneg2 = np.array([[2, 2], [2, 2]])
nonneg3 = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])

neg1 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
neg2 = np.array([[10, 8, 7], [2, 42, 1], [7, 3, 13]])

assert total_positivity_test(pos1)
assert total_positivity_test(pos2)
assert total_positivity_test(pos3)

assert total_non_negativity_test(nonneg1)
assert total_non_negativity_test(nonneg2)
assert total_non_negativity_test(nonneg3)
assert total_non_negativity_test(pos1)
assert total_non_negativity_test(pos2)
assert total_non_negativity_test(pos3)

assert not total_positivity_test(nonneg1)
assert not total_positivity_test(nonneg2)
assert not total_positivity_test(nonneg3)

assert not total_non_negativity_test(neg1)
assert not total_non_negativity_test(neg2)
assert not total_positivity_test(neg1)
assert not total_positivity_test(neg2)

# Функция для подсчета вполне положительных матриц размера nxn с элементами от 1 до d

In [80]:
def count_matrices(n: int, d: int):
    def dfs(i: int, j: int, mat: np.ndarray):
        cnt = 0

        for m in range(1, d + 1):
            mat[i, j] = m

            if not check_minor(mat, i, j):
                continue

            if i == j == n - 1:
                cnt += 1
                continue

            if i == n - 1:
                cnt += dfs(j + 1, j + 1, mat)
                continue

            if j == n - 1:
                cnt += dfs(i + 1, i, mat)
                continue

            if i > j:
                cnt += dfs(i + 1, j, mat)
            else:
                cnt += dfs(i, j + 1, mat)

        return cnt

    mat = np.zeros(shape=(n, n))

    return dfs(0, 0, mat)

# Функция для красивого вывода таблицы на экран

In [81]:
def matrix_pretty_print(mat: np.ndarray):
    for i in range(mat.shape[0]):
        line = ''

        for j in range(mat.shape[1]):
            line += f'{int(mat[i, j]):12}'

        print(line)


# Пересчёт самой таблицы

In [82]:
from IPython.display import clear_output

excluded = [(4, 6), (4, 7), (5, 5), (5, 6), (5, 7), (6, 4), (6, 5), (6, 6), (6, 7)]

counts = np.full(shape=(6, 7), fill_value=-1)

for n in range(1, 7):
    for d in range(1, 8):
        if (n, d) in excluded:
            continue

        counts[n - 1, d - 1] = count_matrices(n, d)

        clear_output(wait=True)
        matrix_pretty_print(counts)

           1           2           3           4           5           6           7
           0           5          33         112         288         605        1145
           0           0          22         597        7178       43090      207494
           0           0           0           0         156          -1          -1
           0           0           0           0          -1          -1          -1
           0           0           0          -1          -1          -1          -1
